# Project for Housing_Price table
- At first created container in microsoft azure storage account 
- Uploaded csv file in the container
- Then integrated the Azure container to databricks 
- In data ingestion then created table from ADLS Gen2 and the table created in cat-edw-training catalog schema is default saved in delta format
- Performed medillian approach to the delta table
- Created gold tables of different scenarios  after cleansing the table



In [0]:
%sql
select * from `cat-edw-training`.default.housing_prices
--the data that doesn’t fit the expected schema is placed into this _rescued_data column so it’s not lost

In [0]:
%sql
DESCRIBE DETAIL `cat-edw-training`.default.housing_prices;


In [0]:
bronze_df = spark.table("`cat-edw-training`.default.housing_prices")




In [0]:
bronze_df.printSchema()
bronze_df.show(5, truncate=False)


In [0]:
from pyspark.sql.functions import col

silver_df = bronze_df \
    .withColumnRenamed("No. of Rooms", "No_of_Rooms") \
    .withColumnRenamed("No. of Bathrooms", "No_of_Bathrooms") \
    .withColumnRenamed("Lot Size", "Lot_Size") \
    .withColumn("SalePrice", col("SalePrice").cast("double")) \
    .withColumn("No_of_Rooms", col("No_of_Rooms").cast("int")) \
    .withColumn("No_of_Bathrooms", col("No_of_Bathrooms").cast("double")) \
    .withColumn("Lot_Size", col("Lot_Size").cast("int")) \
    .filter((col("SalePrice").isNotNull()) & (col("SalePrice") > 0)) \
    .filter((col("No_of_Rooms").isNotNull()) & (col("No_of_Rooms") >= 0)) \
    .filter((col("No_of_Bathrooms").isNotNull()) & (col("No_of_Bathrooms") >= 0))


In [0]:
silver_df.printSchema()
silver_df.display(5, truncate=False)

In [0]:
silver_df.createOrReplaceTempView("silver_housing_prices")#because the object type to gold layer is only views. For bronze and silver the object type is tables


In [0]:
%sql
select * from silver_housing_prices

# Going to Gold_Layer
### # Aggregating the housing_prices table

- Most expensive areas	Top N locations by average price
- Affordability	Avg price per room across locations
- Distribution	Bucket prices into ranges (e.g. <100k, 100k-300k, etc.)
- Data quality	Flag records with suspicious values (e.g., 10+ bathrooms, 0 rooms)

In [0]:
%sql

CREATE OR REPLACE TABLE `cat-edw-training`.default.gold_avg_price_by_location AS
SELECT 
    Location,
     AVG(SalePrice) AS AvgSalePrice
FROM 
    silver_housing_prices
GROUP BY 
    Location



In [0]:
%sql
SELECT current_catalog(), current_schema();
--SHOW SCHEMAS IN cat_edw_training;



In [0]:
%sql
CREATE OR REPLACE TABLE `cat-edw-training`.default.gold_avg_price_per_room AS
SELECT 
avg(SalePrice) AS avg_sale_price, no_of_rooms, Location
from silver_housing_prices
group by no_of_rooms,Location
order by avg(SalePrice) desc


In [0]:
%sql
CREATE OR REPLACE TABLE `cat-edw-training`.default.gold_dist_price_range AS
SELECT 
    CASE
        WHEN `SalePrice` > 1000000 THEN 'Expensive'
        WHEN `SalePrice` < 1000000 THEN 'Affordable'
        ELSE 'Average'
    END AS Price_Label,
    COUNT(*) AS Count
FROM 
    silver_housing_prices --taking columns from silver_housing_prices
GROUP BY 
    CASE
        WHEN `SalePrice` > 1000000 THEN 'Expensive'
        WHEN `SalePrice` < 1000000 THEN 'Affordable'
        ELSE 'Average'
    END
ORDER BY 
    Price_Label; --ordering by Price_Label


In [0]:
%sql
--displays all range count
-- Affordable
CREATE OR REPLACE TABLE `cat-edw-training`.default.gold_dist_allprice_range AS
SELECT 'Affordable' AS Price_Label, COUNT(*) AS Count
FROM silver_housing_prices
WHERE SalePrice < 1000000

UNION ALL

-- Average
SELECT 'Average' AS Price_Label, COUNT(*) AS Count
FROM silver_housing_prices
WHERE SalePrice = 1000000

UNION ALL

-- Expensive
SELECT 'Expensive' AS Price_Label, COUNT(*) AS Count
FROM silver_housing_prices
WHERE SalePrice > 1000000;


In [0]:
%sql
SHOW TABLES IN `cat-edw-training`.default;


In [0]:
avg_price_by_location_df = spark.table("`cat-edw-training`.default.gold_avg_price_by_location")
avg_price_per_room_df = spark.table("`cat-edw-training`.default.gold_avg_price_per_room")
dist_allprice_range_df = spark.table("`cat-edw-training`.default.gold_dist_allprice_range")
avg_price_by_location_df.show()
avg_price_per_room_df.display()
dist_allprice_range_df.show()
